Importar librerias libreriales

In [369]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import tensorflow as tf

print('Keras version: ',keras.__version__)

Keras version:  2.3.0


Importar dataset de Bruno

In [490]:
df_X = pd.read_csv("x_bsg5x7-cne.csv", header=None)
df_y = pd.read_csv("y_bsg5x7-cne.csv", header=None)
y_origen = df_y[0]
y_destino = df_y[1]
df_X = pd.concat([df_X, y_origen], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_X, y_destino, test_size=0.38, random_state=69)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
X_train_2 = np.delete(X_train,range(35,51),1)
X_train_reshaped = np.expand_dims(X_train_2, axis=2)
print(X_train.shape,X_train_reshaped.shape, y_train.shape)

(5141, 51) (5141, 35, 1) (5141,)


In [491]:
X_train_reshaped.shape = (5141, 5, 7) # 5,y son las dimensiones de la bahía
X_train_reshaped[0]
XX_train = X_train[0:,35:50]
XX_train.shape

(5141, 15)

In [496]:
# se hace un swap de la columna de origen con la columna 0 --> esto para facilitar el trabajo de la red
for i in range(len(X_train_reshaped)):
    origin = int(X_train[i][-1])
    aux = X_train_reshaped[i][0]
    X_train_reshaped[i][0]=X_train_reshaped[i][origin]
    X_train_reshaped[i][origin]=aux
    if y_train[i]==0: y_train[i]=origin

In [497]:
y_train[:20]

array([4, 1, 1, 1, 2, 2, 1, 2, 4, 1, 2, 2, 3, 2, 2, 4, 1, 1, 4, 3],
      dtype=int64)

In [498]:
np.unique(y_train)

array([1, 2, 3, 4], dtype=int64)

In [499]:
from sklearn.preprocessing import OneHotEncoder

In [500]:
encoder = OneHotEncoder()
y_train_coded = encoder.fit_transform(y_train.reshape(-1,1)).toarray() 

In [501]:
y_train_coded[:10]

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

Modelado de la red

In [377]:
convmodel = Sequential()
convmodel.add(Conv1D(filters=10, kernel_size=3, activation='relu', input_shape=(7,1))) # los vectores tienen 7 elementos cada uno
convmodel.add(Conv1D(filters=10, kernel_size=2, activation='relu'))
convmodel.add(Dropout(0.1))
convmodel.add(MaxPooling1D(pool_size=2))

In [503]:
#define two different inputs
input1 = Input((7, 1))
input2 = Input((7, 1))
input3 = Input((7, 1))
input4 = Input((7, 1))
input5 = Input((7, 1))
input6 = Input((15, ))
   
#use the model to get three different outputs (the same model)
out1 = convmodel(input1)
out2 = convmodel(input2)
out3 = convmodel(input3)
out4 = convmodel(input4)
out5 = convmodel(input5)

#concatenate the outputs and add the final part of your model: 
out = Concatenate()([out1,out2,out3,out4,out5])
out = Flatten()(out)
out = Concatenate()([out,input6])  
out = Dense(50, activation='relu')(out)
out = Dense(50, activation='relu')(out)
out = Dense(4, activation='softmax')(out)   

#create the model taking 2 inputs with one output
model2 = Model([input1,input2,input3,input4,input5,input6],out)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.save_weights('model.h5')
model2.summary()
#model.save_weights('model_destino.h5')


Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_127 (InputLayer)          (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_128 (InputLayer)          (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_129 (InputLayer)          (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_130 (InputLayer)          (None, 7, 1)         0                                            
___________________________________________________________________________________________

In [504]:
X_train1 = X_train_reshaped[0:,0]

print(X_train1.shape)

X_train2 = X_train_reshaped[0:,1]
X_train3 = X_train_reshaped[0:,2]
X_train4 = X_train_reshaped[0:,3]
X_train5 = X_train_reshaped[0:,4]

X_train1.shape=(5141, 7, 1)
X_train2.shape=(5141, 7, 1)
X_train3.shape=(5141, 7, 1)
X_train4.shape=(5141, 7, 1)
X_train5.shape=(5141, 7, 1)

#model.load_weights('model.h5')
model2.fit([X_train1,X_train2,X_train3,X_train4,X_train5,XX_train], y_train_coded, epochs=200, batch_size=64, verbose=True)

(5141, 7)
Epoch 1/200
5141/5141 [==============================] - 1s 197us/step - loss: 1.3166 - accuracy: 0.3832
Epoch 2/200
5141/5141 [==============================] - 0s 63us/step - loss: 1.1154 - accuracy: 0.5392
Epoch 3/200
5141/5141 [==============================] - 0s 62us/step - loss: 1.0224 - accuracy: 0.5797
Epoch 4/200
5141/5141 [==============================] - 0s 60us/step - loss: 0.9844 - accuracy: 0.6036
Epoch 5/200
5141/5141 [==============================] - 0s 59us/step - loss: 0.9624 - accuracy: 0.6061
Epoch 6/200
5141/5141 [==============================] - 0s 69us/step - loss: 0.9500 - accuracy: 0.6123
Epoch 7/200
5141/5141 [==============================] - 0s 61us/step - loss: 0.9353 - accuracy: 0.6176
Epoch 8/200
5141/5141 [==============================] - 0s 68us/step - loss: 0.9235 - accuracy: 0.6199
Epoch 9/200
5141/5141 [==============================] - 0s 64us/step - loss: 0.9206 - accuracy: 0.6203
Epoch 10/200
5141/5141 [=============================

In [344]:
def compactState(yard):
    sort = []
    for stack in yard:
      for container in stack:
        if not container in sort:
          sort.append(container)
    sort = sorted(sort)
    maxValue = 0
    for i in range(len(yard)):
      for j in range(len(yard[i])):
        yard[i][j] = sort.index(yard[i][j]) + 1
        if yard[i][j] > maxValue:
          maxValue = yard[i][j]
    return yard

def elevateState(yard, h, max_item):
    for stack in yard:
      while len(stack) < h:
        #stack.insert(0,max_item)
        stack.insert(0,1)
    return yard

def flattenState(state):
    flatten = []
    for lista in state:
        for item in lista:
            flatten.append(item)
    return flatten

def normalize(state,max_item):
    return np.array(state)/max_item


def getStackValues(yard): #sorted stacks?
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if i==0:
                cont += 1
            else:
                if stack[i] <= stack[i-1]:
                    cont += 1
                else: break
        values.append(cont)
    return values

def getStackValues_old(yard):
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if flag:
                cont += 1
            else:
                if i == 0:
                    continue
                if stack[i] > stack[i-1]:
                    cont += 1
                    flag = True
        values.append(cont)
    return values

def getStackLen(yard):
    lens = []
    for stack in yard:
        lens.append(len(stack))
    return lens

def getTopStacks(yard,max_item):
    tops = []
    for stack in yard:
        if len(stack) != 0:
            tops.append(stack[len(stack)-1])
        else:
            tops.append(max_item)
    return tops


In [342]:
def generate_ann_state(yard,H):
    max_item = max([max(stack) for stack in yard])
    stackValues=getStackValues(yard) #well-placed
    stacksLen = getStackLen(yard)
    topStacks = getTopStacks(yard,max_item)
    print("stackValues:",stackValues)
    print("stacksLen:",stacksLen)
    print("topStacks:",topStacks)
    
    print("yard:\n",yard)
    yard=elevateState(yard,H, max_item)
    print("elevateState:\n",yard)
    yard=compactState(yard)
    print("compactState:\n",yard)
    yard=flattenState(yard)
    print("flattenState:\n",yard)
    yard=normalize(yard,max_item)
    print("normalize:\n",yard)
    state=yard
    state=np.append(state,normalize(stackValues,H))
    state=np.append(state,normalize(stacksLen,H))
    state=np.append(state,normalize(topStacks,max_item))
    print("state:\n",state)
    return state

In [343]:
yard = [[2, 22, 14, 20, 8], [9, 1, 19, 21, 5], [7, 24, 3, 17, 6], [15, 18, 16, 12, 11], [25, 4, 13, 10, 23]]
preprocess_data(yard,7)

stackValues: [1, 2, 1, 1, 2]
stacksLen: [5, 5, 5, 5, 5]
topStacks: [8, 5, 6, 11, 23]
yard:
 [[2, 22, 14, 20, 8], [9, 1, 19, 21, 5], [7, 24, 3, 17, 6], [15, 18, 16, 12, 11], [25, 4, 13, 10, 23]]
elevateState:
 [[25, 25, 2, 22, 14, 20, 8], [25, 25, 9, 1, 19, 21, 5], [25, 25, 7, 24, 3, 17, 6], [25, 25, 15, 18, 16, 12, 11], [25, 25, 25, 4, 13, 10, 23]]
compactState:
 [[25, 25, 2, 22, 14, 20, 8], [25, 25, 9, 1, 19, 21, 5], [25, 25, 7, 24, 3, 17, 6], [25, 25, 15, 18, 16, 12, 11], [25, 25, 25, 4, 13, 10, 23]]
flattenState:
 [25, 25, 2, 22, 14, 20, 8, 25, 25, 9, 1, 19, 21, 5, 25, 25, 7, 24, 3, 17, 6, 25, 25, 15, 18, 16, 12, 11, 25, 25, 25, 4, 13, 10, 23]
normalize:
 [1.   1.   0.08 0.88 0.56 0.8  0.32 1.   1.   0.36 0.04 0.76 0.84 0.2
 1.   1.   0.28 0.96 0.12 0.68 0.24 1.   1.   0.6  0.72 0.64 0.48 0.44
 1.   1.   1.   0.16 0.52 0.4  0.92]
state:
 [1.         1.         0.08       0.88       0.56       0.8
 0.32       1.         1.         0.36       0.04       0.76
 0.84       0.2        1. 

array([1.        , 1.        , 0.08      , 0.88      , 0.56      ,
       0.8       , 0.32      , 1.        , 1.        , 0.36      ,
       0.04      , 0.76      , 0.84      , 0.2       , 1.        ,
       1.        , 0.28      , 0.96      , 0.12      , 0.68      ,
       0.24      , 1.        , 1.        , 0.6       , 0.72      ,
       0.64      , 0.48      , 0.44      , 1.        , 1.        ,
       1.        , 0.16      , 0.52      , 0.4       , 0.92      ,
       0.14285714, 0.28571429, 0.14285714, 0.14285714, 0.28571429,
       0.71428571, 0.71428571, 0.71428571, 0.71428571, 0.71428571,
       0.32      , 0.2       , 0.24      , 0.44      , 0.92      ])

In [434]:
import copy
def generate_ann_state_old(yard,H):
    yard=copy.deepcopy(yard)
    max_item = max([max(stack) for stack in yard])
    stackValues=getStackValues_old(yard) #well-placed
    stacksLen = getStackLen(yard)
    topStacks = getTopStacks(yard,max_item)
    yard=elevateState(yard,H, max_item)
    yard=compactState(yard)
    yard=flattenState(yard)
    yard=normalize(yard,max_item)
    state=yard
    state=np.append(state,normalize(stackValues,max_item))
    state=np.append(state,normalize(stacksLen,max_item))
    state=np.append(state,normalize(topStacks,max_item))
    return state

In [526]:
layout = px.read("5-5/data5-5-1.dat")
print(layout.stacks)

[[2, 22, 14, 20, 8], [9, 1, 19, 21, 5], [7, 24, 3, 17, 6], [15, 18, 16, 12, 11], [25, 4, 13, 10, 23]]


In [527]:
def predict_origin(layout,model):
    X_test=generate_ann_state_old(layout.stacks,7)
    X1=X_test[0:7]
    X2=X_test[7:14]
    X3=X_test[14:21]
    X4=X_test[21:28]
    X5=X_test[28:35]
    X6=X_test[35:50]
    X1.shape=(1,7,1)
    X2.shape=(1,7,1)
    X3.shape=(1,7,1)
    X4.shape=(1,7,1)
    X5.shape=(1,7,1)
    X6.shape=(1,15)
    return np.argmax(model.predict([X1,X2,X3,X4,X5,X6]))

def predict_destination(layout,model,origin):
    X_test=generate_ann_state_old(layout.stacks,7)
    X1=X_test[0:7]
    X2=X_test[7:14]
    X3=X_test[14:21]
    X4=X_test[21:28]
    X5=X_test[28:35]
    X6=X_test[35:50]
    X1.shape=(1,7,1)
    X2.shape=(1,7,1)
    X3.shape=(1,7,1)
    X4.shape=(1,7,1)
    X5.shape=(1,7,1)
    X6.shape=(1,15)
    input = [X1,X2,X3,X4,X5,X6]
    aux = input[0] 
    input[0] = input[origin] 
    input[origin] = aux
    dest =  np.argmax(model.predict([X1,X2,X3,X4,X5,X6]))+1
    if dest==origin: return 0
    return dest

In [533]:
origin = predict_origin(layout,model)
destination = predict_destination(layout,model2,origin)
layout.move(origin,destination)
layout.stacks

[[2, 22, 14, 20, 8, 23, 21],
 [9, 1, 19],
 [7, 24, 3, 17, 6],
 [15, 18, 16, 12, 11, 5],
 [25, 4, 13, 10]]

In [455]:
import Layout
#layout = px.read("5-5/data5-5-1.dat")
Layout.select_destination_stack(layout,4)

(3, False)

In [456]:
layout.move(4,3)

23

In [441]:
layout.stacks

[[2, 22, 14, 20, 8],
 [9, 1, 19, 21, 5],
 [7, 24, 3, 17, 6],
 [15, 18, 16, 12, 11, 23],
 [25, 4, 13, 10]]

In [457]:
X_test=generate_ann_state_old(layout.stacks,7)
predict(X_test)

array([[5.5190844e-06, 9.0674722e-01, 9.1868438e-02, 5.3845371e-05,
        1.3250117e-03]], dtype=float32)

In [458]:
Layout.select_destination_stack(layout,1)

(4, False)

In [459]:
layout.move(1,4)

5

In [460]:
layout.stacks

[[2, 22, 14, 20, 8],
 [9, 1, 19, 21],
 [7, 24, 3, 17, 6],
 [15, 18, 16, 12, 11, 23],
 [25, 4, 13, 10, 5]]

In [461]:
X_test=generate_ann_state_old(layout.stacks,7)
predict(X_test)

array([[2.5290902e-12, 2.5127270e-07, 2.7419631e-08, 1.8554353e-20,
        9.9999976e-01]], dtype=float32)

In [462]:
Layout.select_destination_stack(layout,4)

(1, False)

In [463]:
layout.move(4,1)

5

In [464]:
X_test=generate_ann_state_old(layout.stacks,7)
predict(X_test)

array([[5.5190844e-06, 9.0674722e-01, 9.1868438e-02, 5.3845371e-05,
        1.3250117e-03]], dtype=float32)

In [534]:
#model.save_weights('model_origen.h5')
#model2.save_weights('model_destino.h5')